In [13]:
import pandas as pd
import numpy as np
df=pd.read_csv("train.csv")
df.head()
df.replace(["male","female"],[0,1],inplace=True)
print(df["Name"].head())

def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'
def title_map(title):
    if title in ['Mr']:
        return 1
    elif title in ['Master']:
        return 3
    elif title in ['Ms','Mlle','Miss']:
        return 4
    elif title in ['Mme','Mrs']:
        return 5
    else:
        return 2
    
df['title'] = df['Name'].apply(get_title).apply(title_map)   

# X["average_montly_hours"] = pd.cut(X["average_montly_hours"], bins=[0,150,200,1000], labels=[1,2,3])

mf = df['Fare'].mean()
df['Fare'] = df['Fare']>mf

# df.loc[ df['Fare'] <= 7.91, 'Fare'] = 0
# df.loc[(df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
# df.loc[(df['Fare'] > 14.454) & (df['Fare'] <= 31), 'Fare'] = 2
# df.loc[ df['Fare'] > 31, 'Fare'] = 3

# # convert from float to int
# df['Fare'] = df['Fare'].astype(int)



df.drop(["Ticket", "Cabin", "Name", "PassengerId", "Parch"],axis="columns" ,inplace=True)
df = pd.get_dummies(df)

print(df.groupby("title").Age.mean())
print(df[df.Age==np.NaN].shape)
df["Age"][df["title"]==1].fillna(32, inplace=True)
df["Age"][df["title"]==2].fillna(45, inplace=True)
df["Age"][df["title"]==3].fillna(4, inplace=True)
df["Age"][df["title"]==4].fillna(21, inplace=True)
print("-"*30)
print(df[df.Age==np.NaN].shape)
print("-"*30)
df["Age"][df["title"]==5].fillna(35, inplace=True)
print("="*30)
print(df[df.Age==np.NaN].shape)
print("="*30)
mage = df.Age.mean()
df.Age = df.Age.fillna(mage)
df["Age"] = pd.cut(df["Age"], bins=[-1,10, 16, 100], labels=[100,10,1])
df['Age'] = df['Age'].astype(int)

# for i in range(1, 6):
#     df["Fare"][df["title"]==i] = df["Fare"][df["title"]==i]>df["Fare"][df["title"]==i].mean()

print(df["Fare"].describe())

df["SibSp"].fillna(0, inplace=True)
df["SibSp"] = pd.cut(df["SibSp"], bins=[-1,2,4,6,100], labels=[8, 10, 3, 2])
# df["SibSp"]
# df["SibSp"][df["SibSp"] != 8]
df['SibSp'] = df['SibSp'].astype(int)


# # print(df.columns)
# # df["fam"] = df["SibSp"] + df["Parch"]
print(df.groupby("Fare")["Survived"].describe())
# # print(df.groupby("Survived")["SibSp"].describe())
# # print(df.groupby("Survived")["Parch"].describe())
# # df.groupby("Survived")["SibSp"].describe()


# # df.describe()
# # df.groupby("Pclass").Age.describe()
# # mn = df.groupby("Survived").mean()
# # v1 = mn.iloc[0,:]
# # v2 = mn.iloc[1,:]
# # (v1-v2)/(v1+v2)
# # df.head()

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

y = df["Survived"]
X = df.drop(["Survived"], axis='columns')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = XGBClassifier()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

yp = model.predict(X_test)
cm = confusion_matrix(yp, y_test)
cm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
model = XGBClassifier()

# n_estimators = [int(x) for x in np.linspace(start=1, stop=100, num = 20)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(1, 50, num = 11)]
# max_depth.append(None)
# min_samples_split = [2, 3, 5, 10, 12, 15, 20]
# min_samples_leaf = [1, 2, 4,6,8,10]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

random_grid = {'eta': [x for x in np.linspace(start=0.01, stop=0.2, num = 4)],
               'max_depth':range(1,10,1),
               'min_child_weight':range(1,13,2),
               'gamma':[i/10.0 for i in range(0,5)],
             'subsample':[i/100.0 for i in range(60,100, 5)],
             'colsample_bytree':[i/100.0 for i in range(60,100, 5)],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
              }

rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
# rf_random = GridSearchCV(model, random_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
rf_random.fit(X_train, y_train)
print(rf_random.best_score_)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

y = df["Survived"]
X = df.drop(["Survived"], axis='columns')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = XGBClassifier(**rf_random.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

yp = model.predict(X_test)
cm = confusion_matrix(yp, y_test)
cm

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object
title
1    32.368090
2    45.545455
3     4.574167
4    21.845638
5    35.788991
Name: Age, dtype: float64
(0, 10)
------------------------------
(0, 10)
------------------------------
(0, 10)
count       891
unique        2
top       False
freq        680
Name: Fare, dtype: object
       count      mean       std  min  25%  50%  75%  max
Fare                                                     
False  680.0  0.317647  0.465904  0.0  0.0  0.0  1.0  1.0
True   211.0  0.597156  0.491636  0.0  0.0  1.0  1.0  1.0


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in fu

0.776536312849162


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in fu

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in fu

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in fu

0.815230961298377
0.8268156424581006


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[98, 28],
       [ 3, 50]], dtype=int64)

In [14]:
import pandas as pd
df=pd.read_csv("test.csv")
df.head()
df.replace(["male","female"],[0,1],inplace=True)
df['title'] = df['Name'].apply(get_title).apply(title_map)   
mf = df['Fare'].mean()
df['Fare'] = df['Fare']>mf
PassengerId= df["PassengerId"]
df.drop(["Ticket", "Cabin", "Name", "PassengerId", "Parch"],axis="columns" ,inplace=True)
df = pd.get_dummies(df)
df["Age"][df["title"]==1].fillna(32, inplace=True)
df["Age"][df["title"]==2].fillna(45, inplace=True)
df["Age"][df["title"]==3].fillna(4, inplace=True)
df["Age"][df["title"]==4].fillna(21, inplace=True)
df["Age"][df["title"]==5].fillna(35, inplace=True)
mage = df.Age.mean()
df.Age = df.Age.fillna(mage)
df["Age"] = pd.cut(df["Age"], bins=[-1,10, 16, 100], labels=[100,10,1])
df['Age'] = df['Age'].astype(int)
df["SibSp"].fillna(0, inplace=True)
df["SibSp"] = pd.cut(df["SibSp"], bins=[-1,2,4,6,100], labels=[8, 10, 3, 2])
df['SibSp'] = df['SibSp'].astype(int)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
yp = model.predict(df)
prediction=pd.DataFrame(yp,PassengerId,columns=["Survived"])
prediction.to_csv("out_titanic.csv",index_label=["PassengerId"])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
